In [28]:
import time
import queue
import pandas as pd
import numpy as np

In [29]:
# puzzle size
X = 8

# reads file with data
file = pd.read_csv('../samplePuzzles.txt', sep=" ", header=None)

#reads file with random puzzle
# file = pd.read_csv('50puzzles/50puzzles.txt', sep=" ", header=None)

df = pd.DataFrame(file)
# goal states
goal1 = np.array([1,2,3,4,5,6,7,0])
goal2 = np.array([1,3,5,7,2,4,6,0])

In [ ]:
AVERAGE_COST = []
AVERAGE_SEARCH = []
AVERAGE_TIME = []
NO_SOLUTION = 0

for numberOfRows in range(0,df.shape[0]):
    # row
    row = df.loc[numberOfRows]
    # row in array form
    single = row.values
    
    class Node(object):
        def __init__(self, data):
            self.data = data
            self.children = []
            self.parent = None
            self.cost = 0

        def add_child(self, child, cost):
            child.parent = self
            self.children.append(child)
            child.cost = self.cost + cost

        def get_puzzle(self):
            return self.data

        def get_children(self):
            return self.children

        def get_cost(self):
            return self.cost

        def hamming_distance(self):
            # number of misplaced tiles
            puzzle = self.data
            misplaced_goal1 = X - np.sum(puzzle == goal1)
            misplaced_goal2 = X - np.sum(puzzle == goal2)

            # exclude the blank space (i.e. 0)
            if (puzzle[7] != 0):
                misplaced_goal1 = misplaced_goal1 - 1 
                misplaced_goal2 = misplaced_goal2 - 1

            if (misplaced_goal1 < misplaced_goal2):
                h1 = misplaced_goal1
            else:
                h1 = misplaced_goal2

            return h1
        
        def manhattan_distance(self):
            sol = [[1,2,3,4,5,6,7,0], [1,3,5,7,2,4,6,0]]
            manhattan = [0,0]
            board = self.get_puzzle().tolist()
            for s in range(0,len(sol)):
                for i in board:
                   manhattan[s] += abs(board.index(i) - sol[s].index(i))
            return min(manhattan)
            
        def naive_heuristic(self):
            puzzle = self.data
            if (np.sum(puzzle == goal1) == X):
                return 0
            elif (np.sum(puzzle == goal2) == X):
                return 0
            else:
                return 1

        def get_totalh(self):
            return self.cost + self.hamming_distance()
        
        def get_totalm(self):
            return self.cost + self.manhattan_distance()
        
        def get_totaln(self):
            return self.cost + self.naive_heuristic()
            

        def root_path(self):
            ls = []
            ls.append(self)
            node = self
            while (node.parent != None):
                ls.append(node.parent)
                node = node.parent
            return ls
            

    def swapping(node, puzzle, swap_position, position, cost): 
            swap_puzzle = np.zeros(8, dtype=np.int16)
            np.copyto(swap_puzzle, puzzle)

            temp = swap_puzzle[swap_position]
            swap_puzzle[swap_position] = swap_puzzle[position]
            swap_puzzle[position] = temp

            swap_child = Node(swap_puzzle)
            node.add_child(swap_child, cost)


    def actions(node):
        # movements 
        puzzle = node.get_puzzle()
        position = puzzle.tolist().index(0)

        if (position != 0 and position != 4):
            # swap left
            swap_position = position - 1
            cost = 1
            swapping(node, puzzle, swap_position, position, cost)

        if (position != 3 and position != 7):
            # swap right
            swap_position = position + 1
            cost = 1
            swapping(node, puzzle, swap_position, position, cost)

        if (position > 3):
            # swap up
            swap_position = position - 4
            cost = 1
            swapping(node, puzzle, swap_position, position, cost)

        else:
            # swap down
            swap_position = position + 4
            cost = 1
            swapping(node, puzzle, swap_position, position, cost)

        if (position == 0):
            # wrapping
            swap_position = 3
            cost = 2
            swapping(node, puzzle, swap_position, position, cost)

            # diagonal 
            cost = 3
            swap_position = 5
            swapping(node, puzzle, swap_position, position, cost)
            swap_position = 7
            swapping(node, puzzle, swap_position, position, cost)

        elif (position == 3):
            # wrapping
            swap_position = 0
            cost = 2
            swapping(node, puzzle, swap_position, position, cost)

            # diagonal 
            cost = 3
            swap_position = 4
            swapping(node, puzzle, swap_position, position, cost)
            swap_position = 6
            swapping(node, puzzle, swap_position, position, cost)

        elif (position == 4):
            # wrapping
            swap_position = 7
            cost = 2
            swapping(node, puzzle, swap_position, position, cost)

            # diagonal 
            cost = 3
            swap_position = 1
            swapping(node, puzzle, swap_position, position, cost)
            swap_position = 3
            swapping(node, puzzle, swap_position, position, cost)

        elif (position == 7):
            # wrapping
            swap_position = 4
            cost = 2
            swapping(node, puzzle, swap_position, position, cost)

            # diagonal
            cost = 3
            swap_position = 0
            swapping(node, puzzle, swap_position, position, cost)
            swap_position = 2
            swapping(node, puzzle, swap_position, position, cost)


    # create root 
    root = Node(single)


    # Before all else, check if initial state = goal state
    class PQ:
        global open_list 
        open_list = queue.PriorityQueue()

        def add_to_list(self, data):
            data1, data2, data3 = data
            items = []

            to_add = 1

            if (open_list.qsize() == 0):
                # first node in priority queue
                to_add = 0
                open_list.put(data)
            elif (open_list.qsize() > 0):
                # sequential nodes
                for x in range(open_list.qsize()):
                    current = open_list.get()
                    if (np.array_equal(current[2].get_puzzle(), data3.get_puzzle()) and current[0] > data1):
                        # replace
                        to_add = 0
                        items.append(data)
                    elif (np.array_equal(current[2].get_puzzle(), data3.get_puzzle()) and current[0] <= data1):
                        # keep current
                        to_add = 0
                        items.append(current)
                    else: 
                        # keep both
                        items.append(current)

            if (to_add == 1): 
                open_list.put(data)

            for x in range(len(items)):
                 open_list.put(items[x])

            return open_list

    # USING HAMMING DISTANCE HEURISTIC
    def heuristic_choice(root, heuristic):
        global NO_SOLUTION
        pq = PQ()
        counter = 0
        numberOfSearches = 0
        costAmount = 0
        
        if (heuristic == "hamming"):
            total = root.get_totalh()
            
            solution_name = "../Output/" + str(numberOfRows) + "_astar-h1_solution.txt"
            solution_file = open(solution_name, "x")
            solution_file.close()

            search_name = "../Output/" + str(numberOfRows) +  "_astar-h1_search.txt"
            search_file = open(search_name, "x")
            search_file.close()
            
        elif (heuristic == "manhattan"):
            total = root.get_totalm()
              
            solution_name = "../Output/" + str(numberOfRows) + "_astar-h2_solution.txt"
            solution_file = open(solution_name, "x")
            solution_file.close()

            search_name = "../Output/" + str(numberOfRows) +  "_astar-h2_search.txt"
            search_file = open(search_name, "x")
            search_file.close()
        else:
            total = root.get_totaln()
            
            solution_name = "../Output/" + str(numberOfRows) + "_astar-demo_solution.txt"
            solution_file = open(solution_name, "x")
            solution_file.close()

            search_name = "../Output/" + str(numberOfRows) +  "_astar-demo_search.txt"
            search_file = open(search_name, "x")
            search_file.close()
        
        root_data = (total, counter, root)
        counter = counter + 1; 
        ls = pq.add_to_list(root_data)

        # solution found
        found = 0

        t_start = time.time()
        t_end = t_start + 60
        while (time.time() < t_end):
            node = ls.get()
            
            if (heuristic == "hamming"):
                check = node[2].hamming_distance()
            elif (heuristic == "manhattan"):
                check = node[2].manhattan_distance()
            else:
                check = node[2].naive_heuristic()

            sear = open(search_name, "a")
            
            if (heuristic == "hamming"):
                distance = node[2].hamming_distance()
                total = node[2].get_totalh()
            elif (heuristic == "manhattan"):
                distance = node[2].manhattan_distance()
                total = node[2].get_totalm()
            else:
                distance = node[2].naive_heuristic()
                total = node[2].get_totaln()
            
            search_path = "[" + str(total) + " " + str(node[2].get_cost()) + " " + str(distance) + " " + str(node[2].get_puzzle()) + "]"
            sear.write(search_path + "\n")
            numberOfSearches = numberOfSearches + 1
            costAmount = node[2].get_cost()
            sear.close()

            sol = open(solution_name, "a")
            if (check == 0):
                found = 1
                path = node[2].root_path()
                path.reverse()
                for x in range(len(path)):
                    
                    if (heuristic == "hamming"):
                        distance = path[x].hamming_distance()
                        total = path[x].get_totalh()
                    elif (heuristic == "manhattan"):
                        distance = path[x].manhattan_distance()
                        total = path[x].get_totalm()
                    else:
                        distance = path[x].naive_heuristic()
                        total = path[x].get_totaln()
                
                    if (x == 0):
                        cost = 0
                        current = 0
                    else:
                        cost = path[x].get_cost() - path[x-1].get_cost() 
                        board = path[x].get_puzzle().tolist()
                        position = path[x-1].get_puzzle().tolist().index(0)
                        current = board[position]
                    solution_path = "[" + str(cost) + " " + str(current) + " " + str(path[x].get_puzzle()) + "]"
                    sol.write(solution_path + "\n")
                time_elapsed = time.time() - t_start
                time_rounded = round(time_elapsed, 2)
                
                AVERAGE_TIME.append(time_rounded)
                
                sol.write(str((path[len(path)-1]).get_cost()) + ", " + str(time_rounded) + "s")
                sol.close()    
                break;
            else: 
                actions(node[2])
                children =  node[2].get_children()
                for x in range(len(children)):
                    
                    if (heuristic == "hamming"):
                        total = children[x].get_totalh()
                    elif (heuristic == "manhattan"):
                        total = children[x].get_totalm()
                    else:
                        total = children[x].get_totaln()
                    
                    data = (total, counter, children[x])
                    counter = counter + 1
                    ls = pq.add_to_list(data)   

        if (found == 0):
            sol = open(solution_name, "a")
            sol.write("No solution")
            
            NO_SOLUTION = NO_SOLUTION + 1
            numberOfSearches = 0
            costAmount = 0
        
        AVERAGE_SEARCH.append(numberOfSearches)
        AVERAGE_COST.append(costAmount)
        
            
    heuristic_choice(root, "hamming")
    heuristic_choice(root, "manhattan")
    heuristic_choice(root, "naive")

# print(NO_SOLUTION)

# TOTAL_COST = sum(AVERAGE_COST)
# print("TC: " + str(TOTAL_COST))
# print("AC: "+ str(TOTAL_COST/len(AVERAGE_COST)))

# TOTAL_SEARCH = sum(AVERAGE_SEARCH)
# print("TS: " + str(TOTAL_SEARCH))
# print("AS: "+ str(TOTAL_SEARCH/len(AVERAGE_SEARCH)))

# TOTAL_TIME = sum(AVERAGE_TIME)
# print("TT: " + str(TOTAL_TIME))
# print("AT: "+ str(TOTAL_TIME/len(AVERAGE_TIME)))

# AVERAGE_COST.clear()
# AVERAGE_SEARCH.clear()
# AVERAGE_TIME.clear()

    